# iQAE | PurdueX ECE 69501.3 
Final Project submission for the course:
## Applied Quantum Computing III : Algorithms and Software
Team Members:
- Vishal Sharathchandra Bajpe
- Lion


In [4]:
# Import libraries
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit.providers import BaseBackend
from qiskit.providers import Backend
from qiskit.aqua import QuantumInstance, AquaError
from scipy.stats import beta
import math


In [12]:
def calculate_L_max(epsilon,alpha,N_shots,T,ci):
    """Calculate Lmax for Algortihm 1
     
    Args:
        epsilon: target accuracy (epsilon > 0)
        alpha: confidence level
        N_shots: Number of shots
        ci: Chosen confidence interval method, which can be either Clopper-Pearson or Chernoff-Hoeffding
        T: Maximum number of rounds
        
    Returns:
        Lmax: Calculated value of Lmax as per conidence interval method selected
    """
    
    if ci == "Chernoff-Hoeffding" :
        return math.asin(math.pow((2/N_shots)*math.log(2*T/alpha),0.25)) # equation 9 as per paper
    if ci == "Clopper-Pearson":
        return pi/21 # after  numerical calculation
        
    
    

In [18]:
# Alogrithm 1
def IQAE(epsilon, alpha, N_shots, ci):
    """Iterative Quantum Amplitude Estimation Algorithm 1
     
    Args:
        epsilon: target accuracy (epsilon > 0)
        alpha: confidence level
        N_shots: Number of shots
        ci: Chosen confidence interval method, which can be either Clopper-Pearson or Chernoff-Hoeffding
        
    Returns:
        a_sub_l: lower value range of ci
        a_sub_m: upper value range of ci
    """
    #------------------Initializing values------------------#
    
    i = 0 # Initialise iteration count
    k_sub_i = 0 # Initialise power of Q
    up_sub_i = True # Keep track of half plane in FINDKNEXT
    
    theta_l, theta_u = 0, pi/2 # Initialize confidence interval
    
    T = math.log2(pi/8*epsilon) # maximum number of rounds
    
    L_max = calculate_L_max(epsilon,alpha,N_shots,T,ci)
    
    #--------------------Algorithm start--------------------#
    
    while((theta_u - theta_l) > 2*epsilon):
        i = i+1 # Increment Iterator
        k_sub_i_prev,up_sub_i_prev = k_sub_i, up_sub_i_prev # Store previous value
        k_sub_i, up_sub_i = FINDNEXTk(k_sub_i_prev,theta_l, theta_u, up_sub_i_prev)
        K_sub_i = 4*k_sub_i +2
        
        if (K_sub_i > (L_max/epsilon)):
            N = (N_shots*L_max/epsilon/K_sub_i/10) # No overshooting condition
        else:
            N = N_shots
            
        # Missing approximation
    
        # if k_sub_i == k_sub_i_prev: # Missing combination
           
        
        if ci == "Chernoff-Hoeffding":
            epsilon_a_sub_i = math.sqrt((1/2*N)*math.log(2*T/alpha))
            a_sub_i_max = min(1, a_sub_i - epsilon_a_sub_i)
            a_sub_i_min = max(0, a_sub_i - epsilon_a_sub_i)
            
        a_sub_i_max,a_sub_i_min = 0,1
        if ci == "Clopper-Pearson":
            a_sub_i_max = beta.ppf(alpha / 2*T, N*a_sub_i,N*(1-a_sub_i)+1)
            a_sub_i_min = beta.ppf(1 - alpha / 2*T, N*a_sub_i + 1 , N*(1-a_sub_i))
            
        # Missing inversion
        
        theta_l = (int(K_sub_i*theta_l) + theta_i_min) / K_sub_i
        thta_u = (int(K_sub_i*theta_u) + theta_i_max) / K_sub_i
        
        a_sub_l, a_sub_u = math.sin(2 * pi * theta_l)**2, math.sin(2 * pi * theta_u)**2
        
        return a_sub_l , a_sub_u
        

In [20]:
# Algorithm 2
def FINDNEXTk (k_sub_i, theta_l, theta_u, up_sub_i, r=2.0):
    """Implement FindNextK as in Figure 2 to find k_sub_i_next
     
    Args:
        k_sub_i: current power of Q
        theta_l: lower bound for confidence interval
        theta_u: upper bound for confidence interval
        up_sub_i: keep track of upper half plane
        r = Minimal ratio of k/k_sub_i_next
        
    Returns:
        k_sub_i_next: next power of Q
        up_sub_i_next: flag value of half-plane tracker for the interval
    
    """
    
    K_sub_i = 4*k_sub_i + 2 # Current theta factor
    theta_i_min = K_sub_i * theta_l # lower bound for scaled theta
    theta_i_max = K_sub_i * theta_u # upper bound for scaled theta
    
    K_max = (pi/(theta_u - theta_i))
    
    K = K_max - (K_max - 2) % 4 # largest potential candidate of the form 4k +2
    
    while K >= r*K_sub_i:
       
        q = K/K_sub_i # Scaling factor for theta_i_min and theta_i_max
        
        if (((q*theta_i_max)%(2*pi)) <= pi) and (((q*theta_i_main)%(2*pi)) <= pi): # interval in upper half plane
           
            K_sub_i_next = K
            up_sub_i_next = True
            k_sub_i_next = (K_sub_i_next - 2)/4
            
            return (k_sub_i_next, up_sub_i_next)
        
        if (((q*theta_i_max)%(2*pi)) >= pi) and (((q*theta_i_main)%(2*pi)) >= pi): # interval in lower half plane
            
            K_sub_i_next = K
            up_sub_i_next = False
            k_sub_i_next = (K_sub_i_next - 2)/4
            
            return (k_sub_i_next, up_sub_i_next)
        
        K = K - 4
        
        return(k_sub_i, up_sub_i)
            
            